In [1]:
import module_cleaning as cl
import pandas as pd

/opt/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Importation de la BDD

In [2]:
df = pd.read_csv('/Users/f.b/Desktop/Data_Science/Clock_industry/Scripts/Data/watches_database.csv')
liste_colonnes= ['marque','modele','mouvement']
df = cl.suppression(df,liste_colonnes)
df = df.iloc[:, 1:]

# Traitement des valeurs manquantes et mise en forme de la BDD

In [3]:
colonnes = ['matiere_boitier','matiere_bracelet','sexe','diametre',
            'etencheite','matiere_lunette','matiere_verre','boucle',
            'matiere_boucle','rouage','reserve_de_marche']

for col in colonnes:
    df = cl.remplissage(df, col)

In [4]:
df = cl.remplissage_mouvement(df, 'mouvement')
df = cl.remplissage_mat_verre (df, 'matiere_verre')
df = cl.remplissage_mouvement_bis(df, 'mouvement')
df = cl.remplissage_mat_verre(df, 'matiere_verre')
df = cl.remplissage_reserve_marche(df, 'reserve_de_marche')
df = cl.remplissage_reserve_marche_bis(df, 'reserve_de_marche')
df['Complications'] = df['fonctions'].apply(cl.count_functions)

In [5]:
colonnes_a_supp = ['rouage', 'fonctions','id']

liste_colonnes = ['etencheite','matiere_bracelet', 'etat', 'sexe', 'diametre', 'matiere_lunette',
                       'matiere_verre', 'boucle','reserve_de_marche' ]


colonne = 'matiere_boitier'

df = cl.suppression_lignes_vides_suite(df, liste_colonnes, colonnes_a_supp, colonne)

# Recodage et mise en forme des variables 

In [6]:
df = cl.traitement_marque(df,'marque')
df = cl.traitement_modele(df,'modele')
df = cl.traitement_mouvement(df, 'mouvement')
df = cl.traitement_matiere_bracelet(df,'matiere_bracelet')
df = cl.traitement_matiere_boitier(df,'matiere_boitier')
df = cl.traitement_annee_prod(df,'annee_prod')
df = cl.traitement_etat(df, 'etat')
df['prix'] = df['prix'].apply(cl.extraire_elements_avant_euro)
df = cl.traitement_prix(df, 'prix')
df['reserve_de_marche'] = df['reserve_de_marche'].apply(cl.extraire_elements_h)
df = cl.traitement_diametre(df, 'diametre')
df = cl.traitement_etencheite(df, 'etencheite')
df = cl.traitement_matiere_lunette(df, 'matiere_lunette')
df = cl.traitement_matiere_verre(df,'matiere_verre')
df = cl.traitement_matiere_boucle(df, 'matiere_boucle')
df = cl.traitement_boucle(df, 'boucle')
df = cl.traitement_ville(df, 'ville')
df = cl.traitement_complication_date(df,'Complications', 'Date_recup')
df = cl.traitement_sexe(df,'sexe')

/Users/f.b/Desktop/Data_Science/Clock_industry/scripts/cleaning_analysis/module_cleaning.py:551: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colonne] = df[colonne].astype('float')


In [7]:
df.to_csv('/Users/f.b/Desktop/Data_Science/Clock_industry/Scripts/Data/verif.csv')